In [1]:
pip install textblob

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Basics
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV





In [16]:
# Stemming, etc.
import re
import nltk

from nltk import pr
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words("english"))

In [18]:
df = pd.read_csv('twitter_data.csv') #Converts test data into a pandas dataframe
print(df.head())
df.shape

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


(24783, 7)

In [20]:
# Map the 'class' column values to corresponding labels
df['labels'] = df['class'].map({
    0: 0,      # Label for hate speech
    1: 1, # Label for offensive language
    2: 2  # Label for neutral speech
})

# Print the first few rows of the DataFrame to verify the changes
print(df.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  labels  
0  !!! RT @mayasolovely: As a woman you shouldn't...       2  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...       1  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...       1  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...       1  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...       1  


In [22]:
print(df.columns)  # This will show all the column names
print(df.head())

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet', 'labels'],
      dtype='object')
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  labels  
0  !!! RT @mayasolovely: As a woman you shouldn't...       2  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...       1  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...       1  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...       1  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...       1  


In [2]:
def clean(text): #preproccessing function
     text = str(text).lower()
     text = re.sub('\[.*?\]', '', text)
     text = re.sub('https?://\S+|www\.\S+', '', text)
     text = re.sub('<.*?>+', '', text)
     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
     text = re.sub('\n', '', text)
     text = re.sub('\w*\d\w*', '', text)
     text = [word for word in text.split(' ') if word not in stopword]
     text = " ".join(text)
     text = [stemmer.stem(word) for word in text.split(' ')]
     text = " ".join(text)
     return text
df["tweet"] = df["tweet"].apply(clean) #applies preprocessing to "tweet" column of test/train dataset
print(df.head())

NameError: name 'df' is not defined

In [25]:
from textblob import TextBlob

# Function to compute sentiment polarity
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df['sentiment'] = df['tweet'].apply(get_sentiment)
print(df.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  labels  sentiment  
0   rt mayasolov woman shouldnt complain clean ho...       2   0.366667  
1   rt  boy dat coldtyga dwn bad cuffin dat hoe  ...       1  -0.700000  
2   rt urkindofbrand dawg rt  ever fuck bitch sta...       1  -0.300000  
3             rt cganderson vivabas look like tranni       1   0.000000  
4   rt shenikarobert shit hear might true might f...       1   0.075000  


In [28]:


# Assuming df['tweet'] contains the text and df['labels'] contains the class labels

# Convert the data into numpy arrays
x = np.array(df["tweet"])
y = np.array(df["labels"])

# Step 1: CV
cv = CountVectorizer(ngram_range=(1, 3))
x = cv.fit_transform(x)  # Transforms the text data into CV feature matrix




# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [38]:
# Create a binary label for class 0
y_binary = (y_train == 0).astype(int)  # 1 if class 0, else 0

# Initialize a classifier for the binary task 
binary_clf = LogisticRegression(class_weight='balanced', random_state=42) # If one wants to test a different binary classifier model here, this is the code for Logistic Regression. Just replace Logistic Regression with the model of your choosing.
binary_clf.fit(X_train, y_binary)

# Evaluate the binary classifier on the test set
y_binary_pred = binary_clf.predict(X_test)
print("Binary Classifier Report for Class 0 vs. Rest:")
print(classification_report((y_test == 0).astype(int), y_binary_pred))


Binary Classifier Report for Class 0 vs. Rest:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      7714
           1       0.43      0.46      0.45       465

    accuracy                           0.93      8179
   macro avg       0.70      0.71      0.71      8179
weighted avg       0.94      0.93      0.94      8179



In [40]:


# If one wants to test a different multi-class classifier model here, this is the code for the RandomForest. Just replace RandomForest with the model of your choosing.
clf_rf = RandomForestClassifier(
    n_estimators=200, min_samples_split=2,
    min_samples_leaf=1,max_depth=20, class_weight='balanced', random_state=42 #parameters
)
clf_rf.fit(X_train, y_train) 



# Make predictions
# Predict with the main multi-class classifier
y_pred_main = clf_rf.predict(X_test)

# Predict with the binary classifier focused on class 0
y_pred_binary = binary_clf.predict(X_test)

# Override predictions based on the binary classifier
for i in range(len(y_pred_main)):
    if y_pred_binary[i] == 1:  # If binary classifier predicts class 0
        y_pred_main[i] = 0

from sklearn.metrics import classification_report

print("Combined Model Classification Report:")
print(classification_report(y_test, y_pred_main))

Combined Model Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.53      0.46       465
           1       0.96      0.86      0.91      6335
           2       0.68      0.93      0.78      1379

    accuracy                           0.86      8179
   macro avg       0.68      0.77      0.72      8179
weighted avg       0.88      0.86      0.86      8179



In [ ]:
# Parameter tester for classifier
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
best_clf = grid_search.best_estimator_

In [42]:
# This function applies the classifier to every tweet in a dataset and returns a dataframe with the text indices and their hate classifiers. 
def process_file_optimized(dft, name, batch_size=500): #dft takes in a pandas dataframe, name is a string, and batch_size indicates how much rows will be processed in each 
    output_list = []
    num_rows = dft.shape[0]

    for start in range(0, num_rows, batch_size):
        end = min(start + batch_size, num_rows)
        batch = dft["text"][start:end]

        # Vectorize and predict for the batch
        transformed_data = cv.transform(batch)
        rf_preds = clf_rf.predict(transformed_data)
        binary_preds = binary_clf.predict(transformed_data)

        # Combine predictions
        output_list.extend([0 if binary_preds[i] == 1 else rf_preds[i] for i in range(len(rf_preds))])

    # Create a DataFrame with the combined results
    result_df = pd.DataFrame(output_list, columns=['Scores for ' + name])
    return result_df

In [48]:
def scorecalc(result_df): #this function takes a result dataframe and counts the 0s, 1s, and 2s of the dataframe. In other words, it counts the number of tweets in each class and returns the # of tweets from each class
    count_zeros = (df == 0).sum().sum() #0 = hate speech
    count_ones = (df == 1).sum().sum() #1 = offensive language
    count_twos = (df == 2).sum().sum() #2 = normal text
    
    print(f"Number of 0s: {count_zeros}")
    print(f"Number of 1s: {count_ones}")
    print(f"Number of 2s: {count_twos}")




In [6]:
# This is the general structure on how to run a dataset through the classification algorithm. Remember that the DF variable returned from process_file_optimized needs to be the same as the printed dataframe and the variable in the dataframe.
result_df15 = process_file_optimized(dffive, "2015 test")
print(result_dffive)

scorecalc(result_dffive)

NameError: name 'process_file_optimized' is not defined

In [44]:
dfsix = pd.read_csv('2016testdata.csv')
result_df16 = process_file_optimized(dfsix, "2016 test")
print(result_df16)

       Scores for 2016 test
0                         2
1                         2
2                         2
3                         2
4                         2
...                     ...
97558                     2
97559                     2
97560                     2
97561                     2
97562                     2

[97563 rows x 1 columns]


In [50]:
scorecalc(result_df16)

Number of 0s: 581
Number of 1s: 8657
Number of 2s: 88325


In [52]:
dfseven = pd.read_csv('2017testdata.csv')
result_df17 = process_file_optimized(dfseven, "2017 test")
print(result_df17)

        Scores for 2017 test
0                          2
1                          2
2                          2
3                          2
4                          2
...                      ...
100689                     2
100690                     2
100691                     2
100692                     2
100693                     1

[100694 rows x 1 columns]


In [54]:
scorecalc(result_df17)

Number of 0s: 891
Number of 1s: 9246
Number of 2s: 90557


In [56]:
dfeight = pd.read_csv('2018testdata.csv')
result_df18 = process_file_optimized(dfeight, "2018 test")
print(result_df18)

       Scores for 2018 test
0                         2
1                         2
2                         2
3                         2
4                         2
...                     ...
92719                     2
92720                     1
92721                     2
92722                     1
92723                     2

[92724 rows x 1 columns]


In [58]:
scorecalc(result_df18)

Number of 0s: 629
Number of 1s: 8160
Number of 2s: 83935


In [60]:
dfnine = pd.read_csv('2019testdata.csv')
result_df19 = process_file_optimized(dfnine, "2019 test")
print(result_df19)

       Scores for 2019 test
0                         2
1                         2
2                         1
3                         2
4                         2
...                     ...
96621                     2
96622                     2
96623                     2
96624                     2
96625                     1

[96626 rows x 1 columns]


In [62]:
scorecalc(result_df19)

Number of 0s: 879
Number of 1s: 10543
Number of 2s: 85204


In [64]:
dfzero = pd.read_csv('2020testdata.csv')
result_df20 = process_file_optimized(dfzero, "2020 test")
print(result_df20)

       Scores for 2020 test
0                         2
1                         2
2                         2
3                         2
4                         2
...                     ...
97837                     2
97838                     2
97839                     2
97840                     1
97841                     1

[97842 rows x 1 columns]


In [66]:
scorecalc(result_df20)

Number of 0s: 994
Number of 1s: 10339
Number of 2s: 86509
